# Generating Sound & Music with Genetic Algorithms

Genetic Algorithms are a subset of evolutionary computing; biologically inspired optimization with population-based trial and error search. Loosely speaking, a genetic algortihm creates a popultation of entities that can be modified, combined, and evaluated for fitness with respect to an optimization goal. The simplest optimization step works as follows; create a population, check which elements are most fit (best suited to the optimization problem), keep only the most fit, modify and recombine elements to create new element for the next generation, repeat.

### The components:
- genetic representation
- population
- genetic operations: mutation and recombination
- fitness
- selection
- sources of randomness

### Introduction Example:

Genetic algorithms are inspired by natural selection. Here is a simple simulation of natural selection (choose the lab): https://phet.colorado.edu/sims/html/natural-selection/latest/natural-selection_en.html

- note all the ways you can change the genetic operations
- note all the ways you can change the fitness function

## The genetic algorithm in this notebook

### The components in the code:

- genetic representation: each element is a sound setting consisting of 28 MIDI CC values
- population: the population consists of 5 
- genetic operations: 
1) point mutation; randomly change a MIDI CC value
2) interpolation; create a interpolation of a CC control from its values of two settings
3) recombination; create a mix of CC values of two settings
- fitness: human-in-the-loop rating
- selection: pick the best sounds and add some new ones
- sources of randomness: in the mutation, interpolation and recombination as well as adding random new ones.

### Getting to know the code:

1. understand the code below and get an overall idea of what the algorithm looks like, maybe draw a flow chart or take some notes.
2. play around with the optimization loop while changing the modify function: do you see things that could be changed in the genetic representation? what are they and can you change them?
3. write a two step selection; first mutate/modify elements and select the fittest (with some randomness), then recombine them and select the fittest from offspring and parent elements (with some randomness)
4. change the modify function, such that some parts of the genetic representation (the chord sequence) can be fixed (made immutable). Create another human input to be able to set which CC controls are immutable from now on. This is the most complex task and probably requires some thinking, creativity, and experimentation. Take your time!

In [1]:
import partitura as pt
import numpy as np
import mido
import random, string
import matplotlib.pyplot as plt
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [2]:
def randomword(length):
    """
    a random character generator
    """
    letters = string.ascii_lowercase
    return ''.join(random.choice(letters) for i in range(length))

In [3]:
class SoundSetting:
    """
    the SoundSetting class represents a parametrization of a sound.
    """
    def __init__(self, number = 28):
        self.number = number
        self.CC = [mido.Message("control_change",
                                channel=0,
                                control=k, # control number k, can be mapped later as you like
                                value=np.random.randint(128) # initialized random midi values
                               ) for k in range(number)]
        self.id = randomword(10)
    
    def mutate(self):
        cidx = np.random.randint(len(self.CC))
        self.CC[cidx].value =np.random.randint(128)
    
    def interpolate(self, anotherSetting):
        cidx = np.random.randint(len(self.CC))
        a = self.CC[cidx].value 
        b = anotherSetting.CC[cidx].value 
        ab = sorted([a, b])
        self.CC[cidx].value = np.random.randint(ab[0], ab[1]+1)
    
    def join(self, anotherSetting):
        #idx to keep
        idx = np.unique(np.random.randint(0,self.number,int(self.number/2)))
        newSetting = SoundSetting()
        newanotherSetting = SoundSetting()
        for k in range(self.number):
            if k in idx:
                newSetting.CC[k] = self.CC[k]
                newanotherSetting.CC[k] = anotherSetting.CC[k]
            else:
                newSetting.CC[k] = anotherSetting.CC[k]
                newanotherSetting.CC[k] = self.CC[k]
        return newSetting, newanotherSetting

In [4]:
"""
MODIFIERS
"""
def modify(population):
    
    # randomize / mutate some elements
    subpop3 = np.random.choice(population, int(len(population)/3))
    for element in subpop3:
        element.mutate()

    # interpolate some elements
    subpop1 = np.random.choice(population, int(len(population)/3))
    subpop2 = np.random.choice(population, int(len(population)/3))
    for element0, element1 in zip(subpop1, subpop2):
        element0.interpolate(element1)
    
    # join some elements
    subpop1 = np.random.choice(population, int(len(population)/3))
    subpop2 = np.random.choice(population, int(len(population)/3))
    for element0, element1 in zip(subpop1, subpop2):
        elnew1, elnew2 = element0.join(element1)
    
        population.append(elnew1)
        population.append(elnew2)
    
    return population

In [5]:
"""
SELECT
"""
def fitness(setting, port):
    for message in setting.CC:
        port.send(message)
        # print("sending CC msg: ", message)
    # the lower the fitness score the better
    try:
        fit = float(input("rate this sound: "))
    except:
        fit = 10
    # add a small random number for hashing
    fit += np.random.rand(1)[0]
    return fit 

def select(population, port, number):
    pop = {ele.id:ele for ele in population}
    fitness_dict = {fitness(ele, port):ele.id for ele in population}
    sorted_fitness = list(fitness_dict.keys())
    sorted_fitness.sort()
    new_pop = [pop[fitness_dict[k]] for k in sorted_fitness[:number]]
    # debug print
    # print(len(new_pop), len(sorted_fitness), len(population), fitness_dict)
    return new_pop, sorted_fitness

In [6]:
import mido
mido.get_output_names()

['Microsoft GS Wavetable Synth 0']

In [10]:
# don't forget to close your port after usage / before reopening
# output_port.close()

In [8]:
# set the port_name to a loopback device to send MIDI to your DAW
# on Mac you can use the IAC driver
# on Windows you can use this free software: https://www.tobias-erichsen.de/software/loopmidi.html
port_name = 'Microsoft GS Wavetable Synth 0'
output_port = mido.open_output(port_name)

In [9]:
"""
LOOP
"""
population = [SoundSetting() for po in range(5)]

for epoch in range(3): 
    population = modify(population)
    print("number of sounds: ", len(population))
    population, sorted_fitness = select(population, output_port, 3) 
    print(f"Epoch {epoch} best fitness: {sorted_fitness[0]:.4f}")
    # population += [SoundSetting() for po in range(2)] uncomment to add new ones

number of sounds:  7
rate this sound: 
rate this sound: 
rate this sound: 
rate this sound: 
rate this sound: 
rate this sound: 
rate this sound: 
3 7 7 {10.013661669336022: 'ozkkepzegz', 10.355762935472947: 'kaiajspged', 10.204204333648335: 'laxpkkhtrv', 10.383648266050193: 'bqqixmuqrh', 10.607077455686182: 'fnfuwmwlev', 10.575321171414021: 'xjmgenlnje', 10.572463375445514: 'timfgegttu'}
Epoch 0 best fitness: 10.0137
number of sounds:  5
rate this sound: 
rate this sound: 
rate this sound: 
rate this sound: 
rate this sound: 
3 5 5 {10.752976757651746: 'ozkkepzegz', 10.667679448971567: 'laxpkkhtrv', 10.734258960326184: 'kaiajspged', 10.746813200528104: 'rjtzyyirui', 10.148550441244094: 'wbxwccfguj'}
Epoch 1 best fitness: 10.1486
number of sounds:  5
rate this sound: 
rate this sound: 
rate this sound: 
rate this sound: 
rate this sound: 
3 5 5 {10.176053363006236: 'wbxwccfguj', 10.549162672417216: 'laxpkkhtrv', 10.679353545316758: 'kaiajspged', 10.75427224739647: 'ryacfouqes', 10.1570

In [ ]:
# you can use this command to send a single CC message at a specified control number
# to learn midi assignments or debug the connection to your DAW / synth

output_port.send(mido.Message("control_change",
                                channel=0,
                                control=27, 
                                value=10 
                               ))